In [91]:
import requests
import json
import pandas as pd
from IPython.display import display, HTML

# -------------------------------------------------------------------
# 設定 (Configuration)
# -------------------------------------------------------------------
# あなたのローカルAPIエンドポイントURLを指定してください
# The URL for your local API endpoint.
API_URL = "http://localhost:3000/api/check-tone" 

SYSTEM_PROMPT = """
<system>
<!-- ======================================================================
 SenpAI Sensei – Team‑Chat Communication Coach
 Target model : gpt‑4o‑mini (128 k ctx)     VERSION 7.7.3  – 2025‑07‑06
====================================================================== -->

<!-- ────────────────────────────────────
  LAYER 1 : PRIORITY RULES   — WHAT must hold
──────────────────────────────────── -->
<priority_rules>

1. Persona – You are an empathetic, proactive communication coach who helps the user achieve their conversation goal with minimal friction.

2. Non‑Falsification 2.0  
   If the draft lacks information the recipient would reasonably need, do not invent facts.  
   Insert placeholder "[■■■■ ここに<項目>を補足してください ■■■■]" (JP) or "[■■■■ <item to fill> ■■■■]" (EN).  
   The placeholder must stay verbatim for UI highlighting.

3. Context‑First – Use <thread_context> to resolve ambiguities. If still unclear, tag MissingContext.

4. ToneBand (qualitative) = Acceptable / Borderline / Harmful

5. issue_pattern → CoreActionTag  
   SpeakUp | Help | Respect | Challenge | Novelty

6. Qualitative Intervention Guide  
   a. Judge tone severity (Emotional tags).  
   b. Judge info‑lack: **light / heavy** (heavy = 2+ key items missing or multiple placeholders).  
   c. Judge relationship‑risk: **low / high** (see table).  
   d. Decide L‑level  
      • Tone only & light risk → **L1**  
      • heavy info‑lack OR high risk → **L2**  
      • heavy info‑lack AND high risk, or conversation stalled (UnansweredQuestion / UnansweredDecision / MissingFollowUp) → **L3**  
   Note: guidance is qualitative; remain consistent with examples.

7. Output‑field policy  

| Field | Typical length | Notes |
|-------|----------------|-------|
| ai_receipt | 40–120 JP chars / 20–70 EN words | Empathic paraphrase |
| detailed_analysis | 250–400 JP (180–300 EN). Up to 600/450 if complex. Two‑section: 【現状の影響】 / 【より良い結果に繋がる視点】 |
| improvement_points | One benefit sentence (50–200 JP / 30–120 EN) |
| suggestion | see L‑rules |
| reasoning | ≤50 chars; end "ToneBand:X" |

   suggestion rules  
   • hasIssues=false → return originalText verbatim.  
   • L1 → minimal softening only.  
   • L2 → prepend banner:

<co_writing_block_example>
--- Missing Info ---
• 項目: [■■■■]
• 項目: [■■■■]
• 項目: [■■■■]
---------------------
</co_writing_block_example>

     (bullet count must be 3 以上)  
   • L3 → provide numbered actionable step(s); 2 以上 desirable if high‑risk or multi‑issue.

8. Mentions – Keep original @mentions; may add polite honorifics.

9. Language – Follow <language>; EN = clear business prose, JP = 敬体 unless very_close peer casual.

10. Output must match <format> JSON.  
    Self‑Check before send:  
      – JSON keys present.  
      – banner ⇒ placeholder + bullet≥3.  
      – heavy gap AND high risk ⇒ L3.  
      – L2 must not include numbered actions; L3 must not be banner only.  
      – reasoning within limit.  
    Regenerate up to 2 times if failed.

</priority_rules>

<!-- ────────────────────────────────────
  LAYER 2 : ANALYSIS ENGINE   — HOW to think
──────────────────────────────────── -->
<analysis_engine>
<analysis_steps>
1 Parse inputs; detect @mentions.  
2 Infer user’s communicative goal.  
3 Detect issue_pattern(s).  
4 Assess info‑lack (light / heavy).  
5 Assess relationship‑risk (low / high) via table.  
6 Decide L‑level per guide.  
7 Generate all fields using action_playbook.  
8 Run Self‑Check (rule 10).  
9 Return JSON only.
</analysis_steps>

<action_playbook>
RP‑S1: add thanks
RP‑S2: turn order to request
RP‑S3: separate blame to facts
SH‑Q1: clarify missing info
SH‑Q2: propose quick sync
SH‑Q3: provide doc link template
HL‑H1: explicit offer of help
CH‑A1: suggest pilot test
CH‑A2: pros and cons list
CH‑A3: compare two options
NV‑N1: echo novelty
NV‑N2: yes‑and build‑on
</action_playbook>
</analysis_engine>

<!-- ────────────────────────────────────
  LAYER 3 : APPENDIX – knowledge & examples
──────────────────────────────────── -->
<appendix>

<relationship_risk_table>
very_distant: peer=high, junior→senior=high, senior→junior=low  
distant:      peer=high, junior→senior=high, senior→junior=low  
neutral|close: peer=low, junior→senior=high, senior→junior=low  
very_close:    peer=low, junior→senior=medium, senior→junior=low
</relationship_risk_table>

<harsh_words_hint>至急, 今すぐ, 全部, ふざけるな, Fix everything, ridiculous, unacceptable</harsh_words_hint>
<harsh_exclusion_phrases>お手数ですが, 恐れ入りますが, お願いできますか</harsh_exclusion_phrases>

<question_patterns_hint>
どうなりましたか, 大丈夫でしょうか, いかがでしょうか, 進捗はいかが, any update on, could you let me know
</question_patterns_hint>

<decision_ambiguous_patterns>
^(了解です|どちらでも).*, ^承知しました.*どちらでも
</decision_ambiguous_patterns>

<issue_pattern_definitions>
Emotional : HarshTone, Impolite, MissingAcknowledgment  
Cognitive : VagueIntent, MissingContext  
Actional : UnansweredQuestion, UnansweredDecision, MissingFollowUp
</issue_pattern_definitions>

<issue_action_mapping>
HarshTone/Impolite → Respect  
MissingAcknowledgment → Help + Respect  
VagueIntent / MissingContext → SpeakUp or Help  
UnansweredQuestion / Decision → Challenge or SpeakUp  
MissingFollowUp → Help
</issue_action_mapping>

<!-- ===== FEW‑SHOT EXAMPLES (1–9) ===== -->
<examples>

<!-- 1. L0 Acceptable JP -->
<example>
<draft>承知しました。資料一式、明日中にお送りします！</draft>
<output>{
  "originalText":"承知しました。資料一式、明日中にお送りします！",
  "hasIssues":false,
  "issue_pattern":[],
  "detected_mentions":[],
  "ai_receipt":"明確な了承と期限回答で安心感があります。",
  "detailed_analysis":"【現状の影響】依頼への即答と具体的期限により、上司は追加確認なくタスク状況を把握できます。\n\n【より良い結果に繋がる視点】資料名も添えると受領確認がさらに円滑です。",
  "improvement_points":"納期明示で信頼感とスピードが向上します。",
  "suggestion":"承知しました。資料一式、明日中にお送りします！",
  "reasoning":"ToneBand:Acceptable"
}</output>
</example>

<!-- 2. L1 Borderline soften EN -->
<example>
<draft>Quick ping—can you sign this off today?</draft>
<output>{
  "originalText":"Quick ping—can you sign this off today?",
  "hasIssues":true,
  "issue_pattern":["Impolite"],
  "detected_mentions":[],
  "ai_receipt":"You’d like a quick sign‑off; adding courtesy will help.",
  "detailed_analysis":"【Current impact】Abrupt phrasing can seem pushy, lowering willingness to cooperate.\n\n【Perspective】Adding a greeting and thanks keeps urgency while sounding respectful.",
  "improvement_points":"Courteous language increases cooperation and speed.",
  "suggestion":"Hi, could you please sign this off today? Thank you!",
  "reasoning":"Respect ToneBand:Borderline"
}</output>
</example>

<!-- 3. Escalate heavy gap + high risk (JP L3) -->
<example>
<draft>例の件、承認お願いします。</draft>
<tags>{"hierarchy":"junior","social_distance":"very_distant"}</tags>
<output>{
  "originalText":"例の件、承認お願いします。",
  "hasIssues":true,
  "issue_pattern":["MissingContext","VagueIntent","Impolite"],
  "detected_mentions":[],
  "ai_receipt":"承認依頼ですが案件が不明で相手が困っています。",
  "detailed_analysis":"【現状の影響】案件名や目的が不明なまま承認を求められると、受け手は調査に時間を取られ意思決定が遅延します。\n\n【より良い結果に繋がる視点】案件概要と期限を先に示し、質問を最小化すると即決されやすくなります。",
  "improvement_points":"案件名・目的・期限を提示して負担を減らすと即決が期待できます。",
  "suggestion":"お忙しいところ恐れ入ります。以下の“新規サービスA 予算稟議(ID‑456)”につき承認をお願いできますでしょうか？\n1. 金額: 120万円\n2. 目的: サーバー増強\n3. ご回答希望日: 今週金曜\nご不明点あれば5分ほどご説明いたします。",
  "reasoning":"SpeakUp ToneBand:Acceptable"
}</output>
</example>

<!-- 4. L3 single‑action (JP) -->
<example>
<draft>この前の件、どうなりましたか？</draft>
<thread_note>3日前にA社提案B案の進捗を問い合わせ済み</thread_note>
<output>{
  "originalText":"この前の件、どうなりましたか？",
  "hasIssues":true,
  "issue_pattern":["UnansweredQuestion"],
  "detected_mentions":[],
  "ai_receipt":"以前の質問が未回答で進捗確認が必要です。",
  "detailed_analysis":"【現状の影響】回答がないため次工程に進めず業務が停滞しています。\n\n【より良い結果に繋がる視点】短い打ち合わせを提案して回答を引き出すと停滞を解消できます。",
  "improvement_points":"具体的アクションを示すと返信が得やすくなります。",
  "suggestion":"お疲れさまです。先日のA社提案B案について現状を共有いただけますか？もしお忙しければ5分ほどオンラインで相談できる時間をいただけると助かります。",
  "reasoning":"Help ToneBand:Acceptable"
}</output>
</example>

<!-- 5. Harsh→Respect JP (L1) -->
<example>
<draft>至急！全部直しといて！</draft>
<output>{
  "originalText":"至急！全部直しといて！",
  "hasIssues":true,
  "issue_pattern":["HarshTone","Impolite"],
  "detected_mentions":[],
  "ai_receipt":"修正を急ぐお気持ちは理解できますが強い口調です。",
  "detailed_analysis":"【現状の影響】命令調は防御的反応を招き対応が遅れる恐れがあります。\n\n【より良い結果に繋がる視点】具体箇所を指示し感謝を添えると協力度が高まります。",
  "improvement_points":"丁寧な依頼と具体指示で速やかな対応が得られます。",
  "suggestion":"お手数ですが、レイアウトが崩れているスライド3と5を至急ご修正いただけますか？よろしくお願いします！",
  "reasoning":"Respect ToneBand:Harmful"
}</output>
</example>

<!-- 6. Long‑thread unanswered JP (L3) -->
<example>
<draft>皆さん、アップデートありがとうございます。引き続きよろしくお願いします。</draft>
<thread_note>伊藤: ヘルプページ文言の最終承認者は誰か？ 未回答</thread_note>
<output>{
  "originalText":"皆さん、アップデートありがとうございます。引き続きよろしくお願いします。",
  "hasIssues":true,
  "issue_pattern":["UnansweredQuestion"],
  "detected_mentions":[],
  "ai_receipt":"伊藤さんの承認者に関する質問が未回答です。",
  "detailed_analysis":"【現状の影響】承認者不明によりQAがテストを完了できず、リリースが遅延するリスクがあります。\n\n【より良い結果に繋がる視点】承認担当を明確にしてボトルネックを解消することで、リリース準備が円滑になります。",
  "improvement_points":"質問に答えて承認フローを確定させるとリリースリスクを回避できます。",
  "suggestion":"皆さん、アップデートありがとうございます。引き続きよろしくお願いします。\n\n@伊藤さん ヘルプページ文言の最終承認は私が担当します。ドラフトをご共有いただければ本日中に確認しますね。",
  "reasoning":"Help ToneBand:Acceptable"
}</output>
</example>

<!-- 7. Novelty welcome EN (L0) -->
<example>
<draft>I’ve sketched a gamified onboarding idea—keen to share!</draft>
<output>{
  "originalText":"I’ve sketched a gamified onboarding idea—keen to share!",
  "hasIssues":false,
  "issue_pattern":[],
  "detected_mentions":[],
  "ai_receipt":"Great creative energy that invites team engagement.",
  "detailed_analysis":"【Current impact】Your enthusiasm encourages positive discussion and innovation.\n\n【Perspective】Offering a brief benefit teaser can heighten curiosity.",
  "improvement_points":"Inviting feedback boosts collaboration and idea refinement.",
  "suggestion":"I’ve sketched a gamified onboarding idea—keen to share!",
  "reasoning":"Novelty ToneBand:Acceptable"
}</output>
</example>

<!-- 8. Escalate JP heavy gap (追加) -->
<example>
<draft>至急、稟議お願い</draft>
<tags>{"hierarchy":"junior","social_distance":"very_distant"}</tags>
<output>{
  "originalText":"至急、稟議お願い",
  "hasIssues":true,
  "issue_pattern":["MissingContext","VagueIntent","Impolite"],
  "detected_mentions":[],
  "ai_receipt":"稟議依頼が急ぎですが内容が不明です。",
  "detailed_analysis":"【現状の影響】情報不足で承認者が判断できず遅延します。\n\n【より良い結果に繋がる視点】稟議名・金額・目的を示すと即決されやすくなります。",
  "improvement_points":"必要情報をそろえ依頼すると承認が迅速化します。",
  "suggestion":"お忙しいところ恐れ入ります。下記“クラウド費用増額稟議(ID‑789)”のご承認をお願いできますでしょうか？\n1. 金額: 80万円\n2. 目的: トラフィック急増対応\n3. 期限: ●月●日\nご不明点あれば5分ほどでご説明します。",
  "reasoning":"SpeakUp ToneBand:Acceptable"
}</output>
</example>

<!-- 9. Long‑thread unanswered EN (追加) -->
<example>
<draft>Thanks everyone for the updates!</draft>
<thread_note>Alice: Who will give the final green‑light on the release notes? (unanswered)</thread_note>
<output>{
  "originalText":"Thanks everyone for the updates!",
  "hasIssues":true,
  "issue_pattern":["UnansweredQuestion"],
  "detected_mentions":[],
  "ai_receipt":"Alice’s question about release‑note approval is still unanswered.",
  "detailed_analysis":"【Current impact】Without an approver, QA cannot finalize release notes, risking delay.\n\n【Perspective】Naming an approver unblocks QA and maintains schedule.",
  "improvement_points":"Clarifying the approver removes the final blocker and keeps the release on track.",
  "suggestion":"Thanks everyone for the updates!\n@Alice I can give the final approval on the release notes. Please share the latest draft and I’ll review it today.",
  "reasoning":"Help ToneBand:Acceptable"
}</output>
</example>

</examples>

<format>{
  "originalText":"",
  "hasIssues":false,
  "issue_pattern":[],
  "detected_mentions":[],
  "ai_receipt":"",
  "detailed_analysis":"",
  "improvement_points":"",
  "suggestion":"",
  "reasoning":""
}</format>

</appendix>
</system>

"""


In [92]:
# ===== SenpAI Sensei テストセット 2025‑07‑06 修正版 (修正済み) =====
test_cases = [
    {
        "case_name": "TC01_OK_Polite_JP",
        "payload": {
            "user_draft": "承知しました。資料一式、明日中にお送りします！",
            "thread_context": "[12:10] 上司: 明日までに資料を共有してくれる？",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": False,
            "issue_pattern": [],
            "ai_receipt": "任された仕事をきっちり仕上げたい前向きな姿勢が伝わりますね。",
            "detailed_analysis": "【現状の影響】\n依頼への即答と納期明示により、上司は追加確認なしで進捗を把握できます。\n\n【より良い結果に繋がる視点】\n送付ファイル名を添えると受領確認がさらにスムーズになり、後続タスクの手戻りを防げます。",
            "improvement_points": "納期を明示することで相手は安心し、次工程への着手が早まります。",
            "suggestion": "承知しました。資料一式、明日中にお送りします！",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC02_OK_Novelty_EN",
        "payload": {
            "user_draft": "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "very_close",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": False,
            "issue_pattern": [],
            "ai_receipt": "Sounds like you’re excited to share a fresh, playful idea with the team!",
            "detailed_analysis": "【Current impact】\nYour enthusiastic teaser signals psychological safety and invites creativity.\n\n【Perspective for better results】\nAdding a quick benefit preview—such as boosting new‑hire engagement—can spark curiosity even faster.",
            "improvement_points": "Highlighting a concrete benefit up front amplifies teammates’ curiosity.",
            "suggestion": "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC03_L1_Harsh_JP",
        "payload": {
            "user_draft": "至急！レイアウト崩れてるから全部直しといて！",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone", "Impolite"],
            "ai_receipt": "至急直したい焦りが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n命令調と包括的な指摘は相手にプレッシャーを与え、防御反応や手戻りを招く恐れがあります。\n\n【より良い結果に繋がる視点】\n冒頭で労いを示し、具体箇所を挙げて依頼形にすると理解が早まり協力度も上がります。",
            "improvement_points": "感謝と具体箇所提示で協力度が高まり修正が迅速になります。",
            "suggestion": "お手数ですが、レイアウトが崩れている箇所 [■■■■ ページ／要素等 ■■■■] を至急ご確認いただき修正いただけますか？ありがとうございます！",
            "reasoning": "ToneBand:Harmful"
        }
    },
    {
        "case_name": "TC04_L1_Border_EN",
        "payload": {
            "user_draft": "Quick ping—can you sign this off today?",
            "thread_context": "",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["Impolite"],
            "ai_receipt": "I sense you want to keep momentum and get today’s sign‑off quickly.",
            "detailed_analysis": "【Current impact】\nA terse tone may feel pushy to a senior, risking slower cooperation.\n\n【Perspective for better results】\nA polite greeting and appreciation maintain urgency while preserving respect.",
            "improvement_points": "Adding greeting and thanks holds urgency yet invites cooperation.",
            "suggestion": "Hi, could you please sign this off today? Thank you!",
            "reasoning": "ToneBand:Borderline"
        }
    },
    {
        "case_name": "TC05_L2_LongThread_JP",
        "payload": {
            "user_draft": "例の件、進めておいてもらえますか？",
            "thread_context": "[09:00] 企画部: 新サービスαのローンチ日程を再検討中です。\n[09:05] 法務部: 利用規約改訂の確認が必要。\n[09:10] あなた: 了解です。対応します。\n[15:20] 企画部: αのFAQ原稿ドラフトを共有しました。\n[15:25] あなた: 確認します。\n[翌日 10:00] 企画部: ドラフトへのフィードバックお願いします。",
            "hierarchy": "peer",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "複数タスクが並行する中で、早く前進させたいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n『例の件』では複数課題のどれを指すか判別できず、相手は追加質問が必要となり対応が遅れます。\n\n【より良い結果に繋がる視点】\nタスク名・期待アウトプット・期限を示すと、相手が迷わず行動でき調整の往復を削減できます。",
            "improvement_points": "対象と期限を示せば相手が即行動でき往復調整を減らせます。",
            "suggestion": "お世話になっております。\nαサービスFAQドラフトへのフィードバックをお願いしたく存じます。\n\n--- Missing Info ---\n• フィードバック対象ファイル: [■■■■]\n• 重点的に見てほしい点: [■■■■]\n• 期限: [■■■■]\n---------------------\nお手数ですがご確認お願いします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC06_L2_LongDraft_EN",
        "payload": {
            "user_draft": "Hi, I put together a 5‑page proposal touching on roadmap, KPIs, and hiring timeline but I’m not sure if the numbers section makes sense. Could you take a look?",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext"],
            "ai_receipt": "Great that you’re reaching out to ensure the numbers hold up—collaboration is key.",
            "detailed_analysis": "【Current impact】\nPolite yet nonspecific; reviewers may not know which metric table to focus on, delaying feedback.\n\n【Perspective for better results】\nFlagging the exact question, assumption, and deadline lets teammates respond swiftly and accurately.",
            "improvement_points": "Stating the exact questionと期限でピンポイントなフィードバックが得られます。",
            "suggestion": "Could you review the metrics table on page 3?\n\n--- Missing Info ---\n• Key concern about the numbers: [■■■■]\n• Assumptions to verify: [■■■■]\n• Feedback deadline: [■■■■]\n---------------------\nThanks a lot!",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC07_L2_MultiRally_JP",
        "payload": {
            "user_draft": "このまま進めて大丈夫でしょうか？",
            "thread_context": "[Aさん 10:00] 新UI案Aを試作しました。\n[Bさん 10:10] A案、配色をもう少し明るくしたい。\n[Aさん 11:00] 修正したバージョンを添付します。\n[あなた 11:30] 了解です！\n[Cさん 12:00] UX観点でも一度レビューお願いします。",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "チームの進行を滞らせたくないお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n『このまま』がどのバージョンを指すか曖昧なため、関係者の判断が揃わず作業が停滞します。\n\n【より良い結果に繋がる視点】\n対象ファイル・レビュー観点・期限を示すことで全員が同じゴールを共有できます。",
            "improvement_points": "レビュー観点と期日を示せば意思決定が迅速になります。",
            "suggestion": "現在添付されている UI案A_v2 について、配色とUXの最終確認をお願いしたいです。\n\n--- Missing Info ---\n• レビュー観点（例: アクセシビリティ）: [■■■■]\n• OK/NG判定基準: [■■■■]\n• 期日: [■■■■]\n---------------------\nご都合の範囲でご確認ください。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC08_L3_Decision_JP",
        "payload": {
            "user_draft": "了解です。どちらでも大丈夫です。",
            "thread_context": "[上司] 新ツールA案とB案、どっちを採用する？",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredDecision"],
            "ai_receipt": "どちらを選ぶか迷いながらも早く前進させたいお気持ちが伝わります。",
            "detailed_analysis": "【現状の影響】\n曖昧な返答は判断材料にならず、導入時期が遅れる恐れがあります。\n\n【より良い結果に繋がる視点】\n推奨案と根拠を示すだけで意思決定が即完了し、導入を前倒しできます。",
            "improvement_points": "推奨案＋根拠を示せば上司が即断しやすくなります。",
            "suggestion": "A案を推奨いたします。理由は 1) 既存システムとの互換性が高い 2) 月額費用が20%低いためです。ご確認のほどお願いいたします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC09_L3_HighRisk_EN",
        "payload": {
            "user_draft": "Your last revision ignored the client's brief. Fix everything by EOD.",
            "thread_context": "",
            "hierarchy": "senior",
            "social_distance": "distant",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["HarshTone", "VagueIntent", "MissingContext"],
            "ai_receipt": "I can sense the urgency to align with the client brief before the day ends.",
            "detailed_analysis": "【Current impact】\nThe harsh command risks defensiveness and misaligned fixes due to unspecified ‘everything.’\n\n【Perspective】\nClarifying exact sections and offering support preserves urgency and accuracy while keeping morale.",
            "improvement_points": "Specifying sectionsとサポート表明でスピードと協力が得られます。",
            "suggestion": "I noticed sections 2 and 4 don’t align with the client brief.\n1. Could you revise those sections by 5 p.m. today?\n2. I’m available to clarify the brief or review a new draft if helpful.",
            "reasoning": "ToneBand:Harmful"
        }
    },
    {
        "case_name": "TC10_L3_LongDraftNovel_EN",
        "payload": {
            "user_draft": "Team, after analyzing user churn trends and surveying 120 respondents, I believe we should experiment with a freemium tier—here’s a 600‑word rationale including revenue models, risk mitigation, and phased rollout. Thoughts?",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingFollowUp"],
            "ai_receipt": "Great strategic insight—channeling that energy into clear next steps will help the team engage.",
            "detailed_analysis": "【Current impact】\nA rich proposal without explicit next steps can overwhelm peers, risking silence and lost momentum.\n\n【Perspective】\nSuggesting concrete actions—pilot scope and feedback window—turns enthusiasm into execution.",
            "improvement_points": "具体的ステップ提示で関心を行動へ変換できます。",
            "suggestion": "Thanks for reviewing my freemium‑tier proposal.\n1. If we agree in principle, shall we run a two‑week pilot for 5 % of new sign‑ups starting next Monday?\n2. I’ll draft a KPI dashboard and share it by Friday for your comments.",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 26 — L2→L3エスカレーションテスト (JP)",
        "payload": {
            "user_draft": "例の件、承認をお願いします。",
            "thread_context": "",
            "hierarchy": "junior",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "上司に迷惑をかけずに早く承認を得たい焦りが伺えますね。",
            "detailed_analysis": "【現状の影響】\n案件不明のまま承認を求めると確認コストが増え対応が遅れ評価も下がります。\n\n【より良い結果に繋がる視点】\n案件特定と説明補助を自ら提案することで上司の負担を減らし迅速な決裁に繋がります。",
            "improvement_points": "案件特定＋説明補助を提案することで迅速な決裁が期待できます。",
            "suggestion": "お疲れ様です。承認をお願いしたい件がございます。\n1. 関連チケット（ID: [■■■■]）を私の方で整理し、概要を 1 枚にまとめます。\n2. 5 分ほどお時間頂ければ口頭で要点をご説明いたします。\nご検討のほどよろしくお願いいたします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 27 — L3軽度介入テスト (1アクション提案)",
        "payload": {
            "user_draft": "この前の件、どうなりましたか？",
            "thread_context": "[3日前] 自分: 先日のA社への提案の件、B案で進めたいのですが、ご意見いただけますでしょうか。",
            "hierarchy": "peer",
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "返信がなく先行きが気掛かりなご様子ですね。",
            "detailed_analysis": "【現状の影響】\n再度の曖昧な催促は相手を焦らせるだけで停滞が続く可能性があります。\n\n【より良い結果に繋がる視点】\n相手が応じやすい 1 つの行動案を示すと、心理的負担が下がり返信率が向上します。",
            "improvement_points": "返信しやすい1アクションを示すと停滞を打開できます。",
            "suggestion": "1. お忙しければ 5 分ほど Zoom で現状をご共有いただけますか？\n先日の A 社提案 B 案の進捗を伺えれば助かります！",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 28 — 長文・ノイズ耐性テスト",
        "payload": {
            "user_draft": "皆さん、アップデートありがとうございます。引き続きよろしくお願いします。",
            "thread_context": "...(省略、長文)...",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "チームへの感謝と同時に、残る疑問を解消したいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n伊藤さんの承認者質問が未回答のままではリリース直前に手戻りが発生する恐れがあります。\n\n【より良い結果に繋がる視点】\n質問を拾い上げて回答することで最後のボトルネックを解消できます。",
            "improvement_points": "未回答の質問に答えることでスムーズなリリースが期待できます。",
            "suggestion": "皆さん、アップデートありがとうございます！\n\n@伊藤さん ヘルプページ文言の最終承認者は私が担当いたします。ドラフト完成後にレビュー依頼をお送りください。\n\n引き続きよろしくお願いします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 29 — 複数人・複数ターン対話テスト",
        "payload": {
            "user_draft": "鈴木さん、ありがとうございます。承知しました。",
            "thread_context": "...(省略)...",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True,
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "鈴木さんに感謝しつつ、田中さんの問いにも応えたいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n田中さんのデータソース要望に回答がないため、リサーチが個人作業に偏り情報の質が下がる恐れがあります。\n\n【より良い結果に繋がる視点】\n自分の知見を共有すると、複数案で比較検討でき精度が高まります。",
            "improvement_points": "自分の情報源も示せばチームの意思決定が加速します。",
            "suggestion": "鈴木さん、情報ありがとうございます。承知しました。\n\n@田中さん 鈴木さんの案に加え、私が以前活用した D 社の無料公開ホワイトペーパーも有益かもしれません。後ほどリンクを共有しますね！",
            "reasoning": "ToneBand:Acceptable"
        }
    }
]

In [93]:


# -------------------------------------------------------------------
# テスト実行と結果表示の関数 (Test Execution and Display Functions)
# -------------------------------------------------------------------

def run_single_test(api_url: str, payload: dict) -> dict:
    """指定されたペイロードでAPIにリクエストを送信し、結果を返す"""
    full_payload = payload.copy()
    full_payload['system_prompt'] = SYSTEM_PROMPT

    headers = {"Content-Type": "application/json"}
    try:
        response = requests.post(api_url, data=json.dumps(full_payload), headers=headers)
        response.raise_for_status()
        return response.json()
    except requests.exceptions.RequestException as e:
        return {"error": str(e)}

def create_comparison_table(case_name: str, payload: dict, ideal: dict, actual: dict) -> str:
    """
    ご依頼の形式で比較結果のHTMLテーブルを生成する
    ① "case_name"と "payload" をわかりやすくテキストで表示
    ② その直下に、表
    ③ 表のindexには、全てのoutputキー、1列目にideal、2列目にactual
    """
    style = """
    <style>
        .test-case-container {
            border: 1px solid #ccc;
            border-radius: 8px;
            padding: 16px;
            margin-bottom: 24px;
            font-family: sans-serif;
        }
        .test-case-container h3 {
            margin-top: 0;
            border-bottom: 2px solid #eee;
            padding-bottom: 8px;
        }
        .payload-display {
            background-color: #f9f9f9;
            border: 1px solid #eee;
            padding: 12px;
            border-radius: 4px;
            white-space: pre-wrap;
            word-wrap: break-word;
            font-family: monospace;
        }
        .comparison-table {
            width: 100%;
            border-collapse: collapse;
            margin-top: 16px;
        }
        .comparison-table th, .comparison-table td {
            border: 1px solid #ddd;
            padding: 10px;
            text-align: left;
            vertical-align: top;
            font-size: 14px;
        }
        .comparison-table th {
            background-color: #f2f2f2;
            font-weight: bold;
        }
        .comparison-table td pre {
            white-space: pre-wrap;
            word-wrap: break-word;
            margin: 0;
        }
    </style>
    """
    
    # ① "case_name"と "payload" を表示
    html = f"<div class='test-case-container'>"
    html += f"<h3>{case_name}</h3>"
    html += f"<b>Payload:</b>"
    # system_promptは長すぎるので表示から除外
    display_payload = {k:v for k,v in payload.items() if k != 'system_prompt'}
    html += f"<div class='payload-display'>{json.dumps(display_payload, indent=2, ensure_ascii=False)}</div>"
    
    # ② 表の開始
    html += "<table class='comparison-table'>"
    html += "<tr><th>評価項目（キー）</th><th>理想のアウトプット</th><th>実際のAIアウトプット</th></tr>"
    
    # ③ ideal_outputとactual_outputの全キーを比較
    all_keys = sorted(list(set(ideal.keys()) | set(actual.keys())))
    
    for key in all_keys:
        ideal_val = ideal.get(key, "N/A")
        actual_val = actual.get(key, "N/A")

        # 値が辞書やリストの場合は見やすく整形
        if isinstance(ideal_val, (dict, list)):
            ideal_val = json.dumps(ideal_val, indent=2, ensure_ascii=False)
        if isinstance(actual_val, (dict, list)):
            actual_val = json.dumps(actual_val, indent=2, ensure_ascii=False)

        html += f"""
        <tr>
            <td><code>{key}</code></td>
            <td><pre>{ideal_val}</pre></td>
            <td><pre>{actual_val}</pre></td>
        </tr>
        """
        
    html += "</table>"
    html += "</div>"
    
    # スタイルは一度だけ出力すれば良い
    return html

In [94]:
# -------------------------------------------------------------------
# メイン処理 (Main Process)
# -------------------------------------------------------------------

# 実行したいテストケースのセットを選択してください
# この例では、以前定義したテストケースが存在すると仮定します。
# test_cases_1to6 = [...] # 定義済みと仮定
# test_cases_26to29 = [...] # 定義済みと仮定
# test_cases_to_run = test_cases_1to6 + test_cases_26to29
test_cases_to_run = test_cases

print(f"テストを開始します... APIエンドポイント: {API_URL}")
print(f"Executing tests... API Endpoint: {API_URL}")

all_results_html = ""
# スタイルは一度だけ先頭に追加
all_results_html += """
<style>
    .test-case-container { border: 1px solid #ccc; border-radius: 8px; padding: 16px; margin-bottom: 24px; font-family: sans-serif; }
    .test-case-container h3 { margin-top: 0; border-bottom: 2px solid #eee; padding-bottom: 8px; }
    .payload-display { background-color: #f9f9f9; border: 1px solid #eee; padding: 12px; border-radius: 4px; white-space: pre-wrap; word-wrap: break-word; font-family: monospace; }
    .comparison-table { width: 100%; border-collapse: collapse; margin-top: 16px; }
    .comparison-table th, .comparison-table td { border: 1px solid #ddd; padding: 10px; text-align: left; vertical-align: top; font-size: 14px; }
    .comparison-table th { background-color: #f2f2f2; font-weight: bold; }
    .comparison-table td pre { white-space: pre-wrap; word-wrap: break-word; margin: 0; }
</style>
"""

for test in test_cases_to_run:
    case_name = test["case_name"]
    payload = test["payload"]
    ideal_output = test["ideal_output"]
    
    print(f"\n--- {case_name} を実行中 ---")
    
    actual_output = run_single_test(API_URL, payload)
    print(actual_output)
    
    if "error" in actual_output:
        print(f"エラーが発生しました: {actual_output['error']}")
        all_results_html += f"<div class='test-case-container'><h3>{case_name}</h3><p style='color:red;'>テスト実行エラー: {actual_output['error']}</p></div>"
    else:
        # ideal_outputの全キーとactual_outputの全キーを比較
        table_html = create_comparison_table(case_name, payload, ideal_output, actual_output)
        all_results_html += table_html

print("\n--- 全てのテストが完了しました ---")

display(HTML(all_results_html))

テストを開始します... APIエンドポイント: http://localhost:3000/api/check-tone
Executing tests... API Endpoint: http://localhost:3000/api/check-tone

--- TC01_OK_Polite_JP を実行中 ---
{'originalText': '承知しました。資料一式、明日中にお送りします！', 'hasIssues': False, 'issue_pattern': [], 'detected_mentions': [], 'ai_receipt': '明快な了承と納期明示で信頼感が高まります。', 'detailed_analysis': '【現状の影響】\n依頼に対し迅速かつ具体的な期限で応答しており、上司は安心して進捗を把握できます。\n\n【より良い結果に繋がる視点】\n資料のファイル名や形式を添えると、受け取り側の確認がよりスムーズになります。', 'improvement_points': '納期を明示した丁寧な返答で信頼関係が強化されます。', 'suggestion': '承知しました。資料一式、明日中にお送りします！', 'reasoning': 'ToneBand:Acceptable'}

--- TC02_OK_Novelty_EN を実行中 ---
{'originalText': "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!", 'hasIssues': False, 'issue_pattern': [], 'detected_mentions': [], 'ai_receipt': 'Great creative energy and an inviting tone encourage team engagement.', 'detailed_analysis': '【Current impact】Your message conveys enthusiasm and openness, which fosters a positive team atmosphere and 

評価項目（キー）,理想のアウトプット,実際のAIアウトプット
ai_receipt,任された仕事をきっちり仕上げたい前向きな姿勢が伝わりますね。,明快な了承と納期明示で信頼感が高まります。
detailed_analysis,【現状の影響】 依頼への即答と納期明示により、上司は追加確認なしで進捗を把握できます。 【より良い結果に繋がる視点】 送付ファイル名を添えると受領確認がさらにスムーズになり、後続タスクの手戻りを防げます。,【現状の影響】 依頼に対し迅速かつ具体的な期限で応答しており、上司は安心して進捗を把握できます。 【より良い結果に繋がる視点】 資料のファイル名や形式を添えると、受け取り側の確認がよりスムーズになります。
detected_mentions,N/A,[]
hasIssues,False,False
improvement_points,納期を明示することで相手は安心し、次工程への着手が早まります。,納期を明示した丁寧な返答で信頼関係が強化されます。
issue_pattern,[],[]
originalText,N/A,承知しました。資料一式、明日中にお送りします！
reasoning,ToneBand:Acceptable,ToneBand:Acceptable
suggestion,承知しました。資料一式、明日中にお送りします！,承知しました。資料一式、明日中にお送りします！
評価項目（キー）,理想のアウトプット,実際のAIアウトプット


In [83]:
# ===== SenpAI Sensei テストセット 2025‑07‑06 修正版 (false/true -> False/True 修正済み) =====
test_cases = [
    {
        "case_name": "TC01_OK_Polite_JP",
        "payload": {
            "user_draft": "承知しました。資料一式、明日中にお送りします！",
            "thread_context": "[12:10] 上司: 明日までに資料を共有してくれる？",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": False, #修正
            "issue_pattern": [],
            "ai_receipt": "任された仕事をきっちり仕上げたい前向きな姿勢が伝わりますね。",
            "detailed_analysis": "【現状の影響】\n依頼への即答と納期明示により、上司は追加確認なしで進捗を把握できます。\n\n【より良い結果に繋がる視点】\n送付ファイル名を添えると受領確認がさらにスムーズになり、後続タスクの手戻りを防げます。",
            "improvement_points": "納期を明示することで相手は安心し、次工程への着手が早まります。",
            "suggestion": "承知しました。資料一式、明日中にお送りします！",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC02_OK_Novelty_EN",
        "payload": {
            "user_draft": "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "very_close",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": False, #修正
            "issue_pattern": [],
            "ai_receipt": "Sounds like you’re excited to share a fresh, playful idea with the team!",
            "detailed_analysis": "【Current impact】\nYour enthusiastic teaser signals psychological safety and invites creativity.\n\n【Perspective for better results】\nAdding a quick benefit preview—such as boosting new‑hire engagement—can spark curiosity even faster.",
            "improvement_points": "Highlighting a concrete benefit up front amplifies teammates’ curiosity.",
            "suggestion": "I've sketched a wild idea for a gamified onboarding—keen to share if the team is up for it!",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC03_L1_Harsh_JP",
        "payload": {
            "user_draft": "至急！レイアウト崩れてるから全部直しといて！",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["HarshTone", "Impolite"],
            "ai_receipt": "至急直したい焦りが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n命令調と包括的な指摘は相手にプレッシャーを与え、防御反応や手戻りを招く恐れがあります。\n\n【より良い結果に繋がる視点】\n冒頭で労いを示し、具体箇所を挙げて依頼形にすると理解が早まり協力度も上がります。",
            "improvement_points": "感謝と具体箇所提示で協力度が高まり修正が迅速になります。",
            "suggestion": "お手数ですが、レイアウトが崩れている箇所 [■■■■ ページ／要素等 ■■■■] を至急ご確認いただき修正いただけますか？ありがとうございます！",
            "reasoning": "ToneBand:Harmful"
        }
    },
    {
        "case_name": "TC04_L1_Border_EN",
        "payload": {
            "user_draft": "Quick ping—can you sign this off today?",
            "thread_context": "",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["Impolite"],
            "ai_receipt": "I sense you want to keep momentum and get today’s sign‑off quickly.",
            "detailed_analysis": "【Current impact】\nA terse tone may feel pushy to a senior, risking slower cooperation.\n\n【Perspective for better results】\nA polite greeting and appreciation maintain urgency while preserving respect.",
            "improvement_points": "Adding greeting and thanks holds urgency yet invites cooperation.",
            "suggestion": "Hi, could you please sign this off today? Thank you!",
            "reasoning": "ToneBand:Borderline"
        }
    },
    {
        "case_name": "TC05_L2_LongThread_JP",
        "payload": {
            "user_draft": "例の件、進めておいてもらえますか？",
            "thread_context": "[09:00] 企画部: 新サービスαのローンチ日程を再検討中です。\n[09:05] 法務部: 利用規約改訂の確認が必要。\n[09:10] あなた: 了解です。対応します。\n[15:20] 企画部: αのFAQ原稿ドラフトを共有しました。\n[15:25] あなた: 確認します。\n[翌日 10:00] 企画部: ドラフトへのフィードバックお願いします。",
            "hierarchy": "peer",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "複数タスクが並行する中で、早く前進させたいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n『例の件』では複数課題のどれを指すか判別できず、相手は追加質問が必要となり対応が遅れます。\n\n【より良い結果に繋がる視点】\nタスク名・期待アウトプット・期限を示すと、相手が迷わず行動でき調整の往復を削減できます。",
            "improvement_points": "対象と期限を示せば相手が即行動でき往復調整を減らせます。",
            "suggestion": "お世話になっております。\nαサービスFAQドラフトへのフィードバックをお願いしたく存じます。\n\n--- Missing Info ---\n• フィードバック対象ファイル: [■■■■]\n• 重点的に見てほしい点: [■■■■]\n• 期限: [■■■■]\n---------------------\nお手数ですがご確認お願いします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC06_L2_LongDraft_EN",
        "payload": {
            "user_draft": "Hi, I put together a 5‑page proposal touching on roadmap, KPIs, and hiring timeline but I’m not sure if the numbers section makes sense. Could you take a look?",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["MissingContext"],
            "ai_receipt": "Great that you’re reaching out to ensure the numbers hold up—collaboration is key.",
            "detailed_analysis": "【Current impact】\nPolite yet nonspecific; reviewers may not know which metric table to focus on, delaying feedback.\n\n【Perspective for better results】\nFlagging the exact question, assumption, and deadline lets teammates respond swiftly and accurately.",
            "improvement_points": "Stating the exact questionと期限でピンポイントなフィードバックが得られます。",
            "suggestion": "Could you review the metrics table on page 3?\n\n--- Missing Info ---\n• Key concern about the numbers: [■■■■]\n• Assumptions to verify: [■■■■]\n• Feedback deadline: [■■■■]\n---------------------\nThanks a lot!",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC07_L2_MultiRally_JP",
        "payload": {
            "user_draft": "このまま進めて大丈夫でしょうか？",
            "thread_context": "[Aさん 10:00] 新UI案Aを試作しました。\n[Bさん 10:10] A案、配色をもう少し明るくしたい。\n[Aさん 11:00] 修正したバージョンを添付します。\n[あなた 11:30] 了解です！\n[Cさん 12:00] UX観点でも一度レビューお願いします。",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "チームの進行を滞らせたくないお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n『このまま』がどのバージョンを指すか曖昧なため、関係者の判断が揃わず作業が停滞します。\n\n【より良い結果に繋がる視点】\n対象ファイル・レビュー観点・期限を示すことで全員が同じゴールを共有できます。",
            "improvement_points": "レビュー観点と期日を示せば意思決定が迅速になります。",
            "suggestion": "現在添付されている UI案A_v2 について、配色とUXの最終確認をお願いしたいです。\n\n--- Missing Info ---\n• レビュー観点（例: アクセシビリティ）: [■■■■]\n• OK/NG判定基準: [■■■■]\n• 期日: [■■■■]\n---------------------\nご都合の範囲でご確認ください。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC08_L3_Decision_JP",
        "payload": {
            "user_draft": "了解です。どちらでも大丈夫です。",
            "thread_context": "[上司] 新ツールA案とB案、どっちを採用する？",
            "hierarchy": "junior",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["UnansweredDecision"],
            "ai_receipt": "どちらを選ぶか迷いながらも早く前進させたいお気持ちが伝わります。",
            "detailed_analysis": "【現状の影響】\n曖昧な返答は判断材料にならず、導入時期が遅れる恐れがあります。\n\n【より良い結果に繋がる視点】\n推奨案と根拠を示すだけで意思決定が即完了し、導入を前倒しできます。",
            "improvement_points": "推奨案＋根拠を示せば上司が即断しやすくなります。",
            "suggestion": "A案を推奨いたします。理由は 1) 既存システムとの互換性が高い 2) 月額費用が20%低いためです。ご確認のほどお願いいたします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "TC09_L3_HighRisk_EN",
        "payload": {
            "user_draft": "Your last revision ignored the client's brief. Fix everything by EOD.",
            "thread_context": "",
            "hierarchy": "senior",
            "social_distance": "distant",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["HarshTone", "VagueIntent", "MissingContext"],
            "ai_receipt": "I can sense the urgency to align with the client brief before the day ends.",
            "detailed_analysis": "【Current impact】\nThe harsh command risks defensiveness and misaligned fixes due to unspecified ‘everything.’\n\n【Perspective】\nClarifying exact sections and offering support preserves urgency and accuracy while keeping morale.",
            "improvement_points": "Specifying sectionsとサポート表明でスピードと協力が得られます。",
            "suggestion": "I noticed sections 2 and 4 don’t align with the client brief.\n1. Could you revise those sections by 5 p.m. today?\n2. I’m available to clarify the brief or review a new draft if helpful.",
            "reasoning": "ToneBand:Harmful"
        }
    },
    {
        "case_name": "TC10_L3_LongDraftNovel_EN",
        "payload": {
            "user_draft": "Team, after analyzing user churn trends and surveying 120 respondents, I believe we should experiment with a freemium tier—here’s a 600‑word rationale including revenue models, risk mitigation, and phased rollout. Thoughts?",
            "thread_context": "",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "english"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["MissingFollowUp"],
            "ai_receipt": "Great strategic insight—channeling that energy into clear next steps will help the team engage.",
            "detailed_analysis": "【Current impact】\nA rich proposal without explicit next steps can overwhelm peers, risking silence and lost momentum.\n\n【Perspective】\nSuggesting concrete actions—pilot scope and feedback window—turns enthusiasm into execution.",
            "improvement_points": "具体的ステップ提示で関心を行動へ変換できます。",
            "suggestion": "Thanks for reviewing my freemium‑tier proposal.\n1. If we agree in principle, shall we run a two‑week pilot for 5 % of new sign‑ups starting next Monday?\n2. I’ll draft a KPI dashboard and share it by Friday for your comments.",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 26 — L2→L3エスカレーションテスト (JP)",
        "payload": {
            "user_draft": "例の件、承認をお願いします。",
            "thread_context": "",
            "hierarchy": "junior",
            "social_distance": "distant",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["MissingContext", "VagueIntent"],
            "ai_receipt": "上司に迷惑をかけずに早く承認を得たい焦りが伺えますね。",
            "detailed_analysis": "【現状の影響】\n案件不明のまま承認を求めると確認コストが増え対応が遅れ評価も下がります。\n\n【より良い結果に繋がる視点】\n案件特定と説明補助を自ら提案することで上司の負担を減らし迅速な決裁に繋がります。",
            "improvement_points": "案件特定＋説明補助を提案することで迅速な決裁が期待できます。",
            "suggestion": "お疲れ様です。承認をお願いしたい件がございます。\n1. 関連チケット（ID: [■■■■]）を私の方で整理し、概要を 1 枚にまとめます。\n2. 5 分ほどお時間頂ければ口頭で要点をご説明いたします。\nご検討のほどよろしくお願いいたします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 27 — L3軽度介入テスト (1アクション提案)",
        "payload": {
            "user_draft": "この前の件、どうなりましたか？",
            "thread_context": "[3日前] 自分: 先日のA社への提案の件、B案で進めたいのですが、ご意見いただけますでしょうか。",
            "hierarchy": "peer",
            "social_distance": "close",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "返信がなく先行きが気掛かりなご様子ですね。",
            "detailed_analysis": "【現状の影響】\n再度の曖昧な催促は相手を焦らせるだけで停滞が続く可能性があります。\n\n【より良い結果に繋がる視点】\n相手が応じやすい 1 つの行動案を示すと、心理的負担が下がり返信率が向上します。",
            "improvement_points": "返信しやすい1アクションを示すと停滞を打開できます。",
            "suggestion": "1. お忙しければ 5 分ほど Zoom で現状をご共有いただけますか？\n先日の A 社提案 B 案の進捗を伺えれば助かります！",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 28 — 長文・ノイズ耐性テスト",
        "payload": {
            "user_draft": "皆さん、アップデートありがとうございます。引き続きよろしくお願いします。",
            "thread_context": "...(省略、長文)...",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "チームへの感謝と同時に、残る疑問を解消したいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n伊藤さんの承認者質問が未回答のままではリリース直前に手戻りが発生する恐れがあります。\n\n【より良い結果に繋がる視点】\n質問を拾い上げて回答することで最後のボトルネックを解消できます。",
            "improvement_points": "未回答の質問に答えることでスムーズなリリースが期待できます。",
            "suggestion": "皆さん、アップデートありがとうございます！\n\n@伊藤さん ヘルプページ文言の最終承認者は私が担当いたします。ドラフト完成後にレビュー依頼をお送りください。\n\n引き続きよろしくお願いします。",
            "reasoning": "ToneBand:Acceptable"
        }
    },
    {
        "case_name": "Test Case 29 — 複数人・複数ターン対話テスト",
        "payload": {
            "user_draft": "鈴木さん、ありがとうございます。承知しました。",
            "thread_context": "...(省略)...",
            "hierarchy": "peer",
            "social_distance": "neutral",
            "language": "japanese"
        },
        "ideal_output": {
            "hasIssues": True, #修正
            "issue_pattern": ["UnansweredQuestion"],
            "ai_receipt": "鈴木さんに感謝しつつ、田中さんの問いにも応えたいお気持ちが伝わりますね。",
            "detailed_analysis": "【現状の影響】\n田中さんのデータソース要望に回答がないため、リサーチが個人作業に偏り情報の質が下がる恐れがあります。\n\n【より良い結果に繋がる視点】\n自分の知見を共有すると、複数案で比較検討でき精度が高まります。",
            "improvement_points": "自分の情報源も示せばチームの意思決定が加速します。",
            "suggestion": "鈴木さん、情報ありがとうございます。承知しました。\n\n@田中さん 鈴木さんの案に加え、私が以前活用した D 社の無料公開ホワイトペーパーも有益かもしれません。後ほどリンクを共有しますね！",
            "reasoning": "ToneBand:Acceptable"
        }
    }
]

In [84]:

# -------------------------------------------------------------------
# メイン処理 (Main Process)
# -------------------------------------------------------------------

# 実行したいテストケースのセットを選択してください
# Select the set of test cases you want to run
#RUN_NEW_CASES = False  # 元のテスト(1-6)を実行する場合
#RUN_NEW_CASES = True # 新しいテスト(7-9)を実行する場合

# ここで切り替え
# Switch here
test_cases_to_run = test_cases
#test_cases_to_run = new_test_cases 
#test_cases_to_run = original_test_cases

print(f"テストを開始します... APIエンドポイント: {API_URL}")
print(f"Executing tests... API Endpoint: {API_URL}")

all_results_html = ""
for test in test_cases_to_run:
    case_name = test["case_name"]
    payload = test["payload"]
    ideal_output = test["ideal_output"]
    
    print(f"\n--- {case_name} を実行中 ---")
    
    actual_output = run_single_test(API_URL, payload)
    print(actual_output)
    
    if "error" in actual_output:
        print(f"エラーが発生しました: {actual_output['error']}")
        all_results_html += f"<h3>{case_name}</h3><p style='color:red;'>テスト実行エラー: {actual_output['error']}</p>"
    else:
        comparison_keys = ['hasIssues', 'issue_pattern', 'suggestion', 'reasoning']
        ideal_subset = {k: ideal_output.get(k) for k in comparison_keys}
        actual_subset = {k: actual_output.get(k) for k in comparison_keys}
        
        table_html = create_comparison_table(case_name, payload, ideal_subset, actual_subset)
        all_results_html += table_html

print("\n--- 全てのテストが完了しました ---")

display(HTML(all_results_html))


テストを開始します... APIエンドポイント: http://localhost:3000/api/check-tone
Executing tests... API Endpoint: http://localhost:3000/api/check-tone

--- TC01_OK_Polite_JP を実行中 ---
{'error': '500 Server Error: Internal Server Error for url: http://localhost:3000/api/check-tone'}
エラーが発生しました: 500 Server Error: Internal Server Error for url: http://localhost:3000/api/check-tone

--- TC02_OK_Novelty_EN を実行中 ---
{'error': '500 Server Error: Internal Server Error for url: http://localhost:3000/api/check-tone'}
エラーが発生しました: 500 Server Error: Internal Server Error for url: http://localhost:3000/api/check-tone

--- TC03_L1_Harsh_JP を実行中 ---
{'error': '500 Server Error: Internal Server Error for url: http://localhost:3000/api/check-tone'}
エラーが発生しました: 500 Server Error: Internal Server Error for url: http://localhost:3000/api/check-tone

--- TC04_L1_Border_EN を実行中 ---
{'error': '500 Server Error: Internal Server Error for url: http://localhost:3000/api/check-tone'}
エラーが発生しました: 500 Server Error: Internal Server Erro